<a href="https://colab.research.google.com/github/brendanpshea/database_class/blob/main/Introduction_to_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Introduction to Structured Query Language
### Brendan Shea, PhD (Brendan.Shea@rctc.edu) | Database and SQL Lecture Notes
This is the first in a set of interactive "lectures" designed to give you some examples of how "Structured Query Language" (SQL) works. Five quick notes about SQL:
1. SQL is a declarative, non-procedural programming language used to interact with relational databases. That is, you'll use SQL to tell the computer "what sort of infomration you want" and NOT "how to get this information." This is very different from procedural languages like Python or Java, and is much more closely related to areas like formal logic or set theory.
2. Like many programming languages, SQL is "standardized" by the American National Standards Institute (ANSI). This ensures that what much of you learn here will be generally applicable regardless of what particular relational database you wnat to work with.
3. SQL is simple! There are are relatively few commands (\< 100) in ANSI standard SQL and the vast majorrity of the work is done by 25 or so.
4. The fun (and challenge!) of SQL comes from trying to put these limited commands to work.You can learn the basics of SQL in 30 minutes. Learning to Wrie a correct, efficient SQL query can be a life's work.
5. These notes (as well as the accompanying labs) are delivered via a Python-based "Jupyter" notebook. You don't need to know any Python to use these notebooks. However, there are a very common way of doing data analysis, statistics, and data science. (So, I'd encourage you to play around with them).

Mainly, we'll be working through a series of sample queries, so you can get a sense of how SQL works.

## 1. Getting the Environment Set Up
Here, we're going to be setting SQLite as our relational database server. You don't really need to worry about the details, but basically we're going to:
1. Install some unix and python utilities allowing us to download files and interact with databases.
2. Download a SQLite database called "movie.sqlite"
3. Lauch a SQLite server and connect to this database.
4. This will all be done automatically--just "run" this cell by hitting "Ctrl-Enter".

SQLite is frequently used for when a single user/application wants to access a database, and is one of the most commonly installed applications in modern computing (you probably have multiple copies on your phone/computer right now--you just don't know it!). 

Once we've started up our SQL server, we'll load the "Movie" database into memory, so we can play with it. (Note: We'll be making a new copy of this database each time we reload the notebook, so don't worry about changing/deleting it when you are doing exercises).

NOTE: Please don't alter the following code cell! I don't expect you to understand the details, but they are necessary to get the notebook ready to run SQL code.

In [1]:
# Some UNIX and Pyhton utilites we need to install for the lab.
!pip install wget --quiet
!pip install sqlalchemy --quiet
!pip install ipython-sql --quiet

# Now let's download the file we'll be using for this lab
!wget 'https://github.com/brendanpshea/database_class/raw/main/movie.sqlite' -q

# Finally, let's make a connnection with the databse
%load_ext sql
%sql sqlite:////content/movie.sqlite

'Connected: @/content/movie.sqlite'

# 2. Getting to Know Our Tables
In this section, we'll get acquainted with the relational schema of our database, and start exploring it using SELECT commands. A SELECT commands is SQL's way of "Reading" data from the database. You'll notice I start each of the code cells with a %% sql, like so:

```
%%sql 
# Write your SQL query here
```
This "tells" the Jupyter notebook I want it to treat everything else in the cell as a SQL command. 

## Examining the Database Schema
Let's begin by taking a look at what tables we have. The command you need here will vary by DBMS (and I won't ask you to do it on a lab/exam), but the general rule is that relational databases will store the "relational schema" for the tables in your database inside their own "system" tables (the documentation will tell you what the names of these tables are). And you can query these tables using SQL! 

Right now, though, I just want you to take a look at what tables we have and at the "CREATE" table statements that were originally used to create them. You'll notice that our (SIMPLE!) table has data types that include:
1. Strings of variable length (VARCHARS)
2. Strings of fixed length (CHARS)
3. Integers (INTEGERS)
4. Dates (DATES)

You'll also notice that our core "strong" tables are Movie and Person. They have their "own" primary keys that aren't foreign keys to other tables. The other tables (ACTOR, DIRECTOR, OSCAR) are "weak" tables that serve to relate these tables in various ways. (E.g., a certain person "acted in" a particular movie but "directed" a different movie).

In [2]:
# This shows table schema. The commands here will depend on the DBMS
%%sql 
SELECT name AS "Table Name", sql AS "Schema of Table" 
  FROM sqlite_master 
  WHERE type = 'table';

 * sqlite:////content/movie.sqlite
Done.


Table Name,Schema of Table
Movie,"CREATE TABLE Movie (id CHAR(7) PRIMARY KEY, name VARCHAR(64), year INTEGER, rating VARCHAR(5), runtime INTEGER, genre VARCHAR(16), earnings_rank INTEGER)"
Person,"CREATE TABLE Person (id CHAR(7) PRIMARY KEY, name VARCHAR(64), dob DATE, pob VARCHAR(128))"
Actor,"CREATE TABLE Actor (actor_id CHAR(7), movie_id CHAR(7), PRIMARY KEY (actor_id, movie_id), FOREIGN KEY (actor_id) REFERENCES Person(id), FOREIGN KEY (movie_id) REFERENCES Movie(id))"
Director,"CREATE TABLE Director(director_id CHAR(7), movie_id CHAR(7), PRIMARY KEY (director_id, movie_id), FOREIGN KEY (director_id) REFERENCES Person(id), FOREIGN KEY (movie_id) REFERENCES Movie(id))"
Oscar,"CREATE TABLE Oscar(movie_id CHAR(7), person_id CHAR(7), type VARCHAR(23), year INTEGER, PRIMARY KEY (person_id, type, year), FOREIGN KEY (movie_id) REFERENCES Movie(id), FOREIGN KEY (person_id) REFERENCES Person(id))"


In [3]:
# Show the first 5 rows of each table
# Don't worry about the python bits here!
movie_df = %sql SELECT * FROM Movie LIMIT 5;
person_df = %sql SELECT * FROM Person LIMIT 5;
actor_df = %sql SELECT * FROM Actor LIMIT 5;
director_df = %sql SELECT * FROM Director LIMIT 5;
oscar_df = %sql SELECT * FROM Oscar LIMIT 5;
print('\nMovie\n', movie_df,
      '\nPerson\n',person_df, 
      '\nActor\n', actor_df, 
      '\nDirector\n', director_df, 
      '\nOscar\n', oscar_df)

 * sqlite:////content/movie.sqlite
Done.
 * sqlite:////content/movie.sqlite
Done.
 * sqlite:////content/movie.sqlite
Done.
 * sqlite:////content/movie.sqlite
Done.
 * sqlite:////content/movie.sqlite
Done.

Movie
 +---------+------------------------------+------+--------+---------+-------+---------------+
|    id   |             name             | year | rating | runtime | genre | earnings_rank |
+---------+------------------------------+------+--------+---------+-------+---------------+
| 2488496 | Star Wars: The Force Awakens | 2015 | PG-13  |   138   |   A   |       1       |
| 4154796 |      Avengers: Endgame       | 2019 | PG-13  |   181   |  AVS  |       2       |
| 0499549 |            Avatar            | 2009 | PG-13  |   162   |  AVYS |       3       |
| 1825683 |        Black Panther         | 2018 | PG-13  |   134   |  AVS  |       4       |
| 4154756 |    Avengers: Infinity War    | 2018 | PG-13  |   149   |  AVYS |       5       |
+---------+------------------------------+-

# 3. Using SELECT, LIMIT, and COUNT
A simple SELECT retrieves EVERY row from a table. It works like this:

```
# Show selected attributes for all rows
SELECT attribute_1, attribute_2, etc. FROM table;
 
# Show all attributes for all rows
SELECT * FROM table; 

# Just show me the first five rows
SELECT * FROM table LIMIT 5;

```

Let's see it in action!

In [41]:
# Let's begin by all attributes for the first five rows of MOVIE
%%sql 

SELECT * 
  FROM Movie 
  LIMIT 5;

 * sqlite:////content/movie.sqlite
Done.


id,name,year,rating,runtime,genre,earnings_rank
2488496,Star Wars: The Force Awakens,2015,PG-13,138,A,1
4154796,Avengers: Endgame,2019,PG-13,181,AVS,2
0499549,Avatar,2009,PG-13,162,AVYS,3
1825683,Black Panther,2018,PG-13,134,AVS,4
4154756,Avengers: Infinity War,2018,PG-13,149,AVYS,5


In [5]:
# Now, we can take a look at the first 7 rows of the Person table
%%sql 
SELECT * FROM Person LIMIT 7;

 * sqlite:////content/movie.sqlite
Done.


id,name,dob,pob
0000002,Lauren Bacall,1924-09-16,"New York, New York, USA"
0000004,John Belushi,1949-01-24,"Chicago, Illinois, USA"
0000006,Ingrid Bergman,1915-08-29,"Stockholm, Sweden"
0000007,Humphrey Bogart,1899-12-25,"New York, New York, USA"
0000008,Marlon Brando,1924-04-03,"Omaha, Nebraska, USA"
0000009,Richard Burton,1925-11-10,"Pontrhydyfen, Wales, UK"
0000010,James Cagney,1899-07-17,"Yonkers, New York, USA"


In [6]:
# COUNT can tell us how many rows are returned.
# This is often more useful than listing the rows out one by one
%%sql 

SELECT COUNT(*) FROM Person;

 * sqlite:////content/movie.sqlite
Done.


COUNT(*)
2627


In [7]:
# We can just list the columns we are interested in. 
# For example, let's just list the name and year of Movies
%%sql

SELECT name, year FROM Movie LIMIT 5;

 * sqlite:////content/movie.sqlite
Done.


name,year
Star Wars: The Force Awakens,2015
Avengers: Endgame,2019
Avatar,2009
Black Panther,2018
Avengers: Infinity War,2018


In [8]:
# If you take a look at the database schema above, you'll notice
# that Actor, Oscar, and Director are JOIN tables.
# They provide different sorts of links between Person and Movie.
# Here's what actor looks like, for example
%%sql

SELECT * FROM Actor LIMIT 6;

 * sqlite:////content/movie.sqlite
Done.


actor_id,movie_id
0000138,0120338
0000701,0120338
0000708,0120338
0000870,0120338
0000200,0120338
0000434,0076759


# 4. Keeping Things DISTINCT
Our simple SELECTS from the previous section returned every single row. Sometimes, though, we just want the unique (of DISTINCT) values. Here are two examples, showing the difference this can make. We'll try to answer the question: "What are the different types of movie ratings (PG, R, etc.) that are in our database?"


In [9]:
# First, let's do it without distinct
%%sql

SELECT rating FROM Movie LIMIT 10;

 * sqlite:////content/movie.sqlite
Done.


rating
PG-13
PG-13
PG-13
PG-13
PG-13
PG-13
PG-13
PG-13
PG-13
PG


In [10]:
# How many rows were returned from that query?
%%sql

SELECT COUNT(rating) FROM Movie;

 * sqlite:////content/movie.sqlite
Done.


COUNT(rating)
590


In [11]:
# Now, we can do with DISTINCT -- notice the difference!
%%sql

SELECT DISTINCT rating FROM Movie;

 * sqlite:////content/movie.sqlite
Done.


rating
PG-13
PG
G
R
NC-17
M
GP
None


In [12]:
# We can again COUNT the rows returned.
%%sql 
SELECT COUNT(DISTINCT rating) FROM Movie;

 * sqlite:////content/movie.sqlite
Done.


COUNT(DISTINCT rating)
7


# 5. Column Aliases and ORDER BY
So far, we've basically let SQL display the rows of our SELECT queries in whatever the "default" setting happens to be. However, for real-world purposes (when we actually want to *use* this data), we often want it to be presented in particular order and format. SQL lets us do this. Here, we'll do the following:
1. Create new column labels (or "aliases") by using the AS keyword
2. Changing the order of rows by using ORDER BY 

In [13]:
# To start, let's take a look at the Movie table
# Notice the results don't seem to be any usable order
%%sql

SELECT id, name, year FROM Movie LIMIT 5;

 * sqlite:////content/movie.sqlite
Done.


id,name,year
2488496,Star Wars: The Force Awakens,2015
4154796,Avengers: Endgame,2019
0499549,Avatar,2009
1825683,Black Panther,2018
4154756,Avengers: Infinity War,2018


In [14]:
# Now, let's trying "renaming" the columns using AS
%%sql 

SELECT id, name AS "Movie Title", 
  year AS "Year Released" 
  FROM Movie LIMIT 5;

 * sqlite:////content/movie.sqlite
Done.


id,Movie Title,Year Released
2488496,Star Wars: The Force Awakens,2015
4154796,Avengers: Endgame,2019
0499549,Avatar,2009
1825683,Black Panther,2018
4154756,Avengers: Infinity War,2018


In [15]:
# OK, now let's trying ordering by the year the movie was Released
%%sql 

SELECT id, name AS "Movie Title", year 
  FROM Movie 
  ORDER BY year 
  LIMIT 5;

 * sqlite:////content/movie.sqlite
Done.


id,Movie Title,year
0018578,Wings,1927
0018379,Seventh Heaven,1927
0018515,Two Arabian Knights,1927
0019071,"Last Command, The",1928
0019729,"Broadway Melody, The",1929


In [16]:
# The default sort order is "Ascending" (or ASC)
# We could also sort by "Descending" order
%%sql 

SELECT id, name, year FROM Movie 
  ORDER BY year DESC 
  LIMIT 10;

 * sqlite:////content/movie.sqlite
Done.


id,name,year
4154796,Avengers: Endgame,2019
6105098,The Lion King,2019
2527338,Star Wars: The Rise Of Skywalker,2019
4520988,Frozen II,2019
1979376,Toy Story 4,2019
4154664,Captain Marvel,2019
6320628,Spider-Man: Far from Home,2019
6139732,Aladdin,2019
7286456,Joker,2019
7975244,Jumanji: The Next Level,2019


In [17]:
# We can also sort by multiple columns
# For example, first descending by year and the ascending by title
%%sql 

SELECT id, name, year FROM Movie 
  ORDER BY year DESC, name ASC 
  LIMIT 10;

 * sqlite:////content/movie.sqlite
Done.


id,name,year
6139732,Aladdin,2019
4154796,Avengers: Endgame,2019
4154664,Captain Marvel,2019
4520988,Frozen II,2019
7349950,It Chapter Two,2019
7286456,Joker,2019
7549996,Judy,2019
7975244,Jumanji: The Next Level,2019
7653254,Marriage Story,2019
7131622,Once Upon a Time... In Hollywood,2019


#6. Calculuated Columns
SQL also allows do things like add, subtract, and divide the values of columns. Here we'll show:
1. How to use SQL to do basic arithemtic
2. How to use this to produce new "calculated columns"

In [18]:
# Let's do some simple arithmetic
%%sql
SELECT 7 + 21

 * sqlite:////content/movie.sqlite
Done.


7 + 21
28


In [19]:
# The order of operations is the same one you should
# have learned in math class
%%sql
SELECT 3 - 2 * 4

 * sqlite:////content/movie.sqlite
Done.


3 - 2 * 4
-5


In [20]:
# The "default" for division is integer division. So, you do NOT get decimal places.
%%sql
SELECT 11/5

 * sqlite:////content/movie.sqlite
Done.


11/5
2


In [21]:
# We can also get the remainder using the % operator
%%sql
select 11/5, 11 % 5

 * sqlite:////content/movie.sqlite
Done.


11/5,11 % 5
2,1


In [22]:
# To do "float" division, we just need to put a . after one of our numbers,
# which tells SQL to treat this as a FLOAT
%%sql 
SELECT 11./5 AS "What is 11 divided by 5?"

 * sqlite:////content/movie.sqlite
Done.


What is 11 divided by 5?
2.2


In [23]:
# Now, let's try our hand at using this to play with movie runtimes
%%sql 
SELECT name, runtime FROM Movie LIMIT 7;

 * sqlite:////content/movie.sqlite
Done.


name,runtime
Star Wars: The Force Awakens,138
Avengers: Endgame,181
Avatar,162
Black Panther,134
Avengers: Infinity War,149
Titanic,194
Jurassic World,124


In [24]:
# How many hours is each movie?
%%sql 
SELECT name, runtime, (runtime/60.) AS "Runtime (hours)" FROM Movie LIMIT 7;

 * sqlite:////content/movie.sqlite
Done.


name,runtime,Runtime (hours)
Star Wars: The Force Awakens,138,2.3
Avengers: Endgame,181,3.0166666666666666
Avatar,162,2.7
Black Panther,134,2.2333333333333334
Avengers: Infinity War,149,2.4833333333333334
Titanic,194,3.2333333333333334
Jurassic World,124,2.066666666666667


In [25]:
# How many seconds is each movie?
%%sql 
SELECT name, runtime, (runtime * 60) AS "Runtime (seconds)" FROM Movie LIMIT 7;

 * sqlite:////content/movie.sqlite
Done.


name,runtime,Runtime (seconds)
Star Wars: The Force Awakens,138,8280
Avengers: Endgame,181,10860
Avatar,162,9720
Black Panther,134,8040
Avengers: Infinity War,149,8940
Titanic,194,11640
Jurassic World,124,7440


#7. Filtering results with WHERE, BETWEEN, and IS (NOT) NULL
So far, we've been focusing on SQL statements of the form SELECT attributes from Table. However, many/most SQL statements will also *filter* these results acccording to whether certain conditions are met. This is where WHERE comes in. For example:


```
SELECT attribute_list FROM table WHERE conditions
```

Below, we'll take a look at how this works.

In [26]:
# Find people in the databse born after 2000
%%sql
SELECT name, dob FROM Person 
  WHERE dob > '2000/1/1'

 * sqlite:////content/movie.sqlite
Done.


name,dob
Richard Glatzer,2015-03-10
Neel Sethi,2003-12-22
Jacob Tremblay,2006-10-05
Sophia Lillis,2002-02-13


In [27]:
# How many movies have a runtime of exactly 120 minutes?
%%sql
SELECT COUNT(*) AS "Movies with runtime of exactly 120 minutes" FROM Movie 
  WHERE runtime = 120

 * sqlite:////content/movie.sqlite
Done.


Movies with runtime of exactly 120 minutes
15


In [28]:
# Or, we could find those with a run time of BETWEEN 60 and 80 minutes
%%sql 
SELECT name, runtime FROM Movie 
  WHERE runtime BETWEEN 60 AND 80

 * sqlite:////content/movie.sqlite
Done.


name,runtime
One Hundred and One Dalmatians,79
"Jungle Book, The",78
Dangerous,79
Morning Glory,74
"Sin of Madelon Claudet, The",75
Min and Bill,66
Coquette,76


In [29]:
# Finally, we can select just those rows have (or do not have) 
# NULL values for particular rows
%%sql
SELECT * FROM Movie 
  WHERE earnings_rank IS NULL 
  LIMIT 5;

 * sqlite:////content/movie.sqlite
Done.


id,name,year,rating,runtime,genre,earnings_rank
0372784,Batman Begins,2005,PG-13,141,AVYT,None
0367594,Charlie and the Chocolate Factory,2005,PG,115,VCFY,None
0382932,Ratatouille,2007,G,110,NCF,None
1981115,Thor: The Dark World,2013,PG-13,112,A,None
0145660,Austin Powers: The Spy Who Shagged Me,1999,PG-13,95,AC,None


# 7. Searching text with LIKE and %
The LIKE operator allows us to search within a text field. In particular:



```
# returns rows where attribute begins with 'a'.
SELECT ... WHERE attribute LIKE 'a%'

# match strings ending with 'a'.
SELECT ... WHERE attribute LIKE '%a' 

# match strings with 'a' anywhere
SELECT ... WHERE attribute LIKE '%a%'
```








In [30]:
# For example, let's find movies that star with the word 'STAR'
%%sql
SELECT * FROM Movie 
  WHERE name LIKE 'Star%';

 * sqlite:////content/movie.sqlite
Done.


id,name,year,rating,runtime,genre,earnings_rank
2488496,Star Wars: The Force Awakens,2015,PG-13,138,A,1
2527336,Star Wars: Episode VIII - The Last Jedi,2017,PG-13,152,AVYS,9
2527338,Star Wars: The Rise Of Skywalker,2019,PG-13,141,AVYS,14
0120915,Star Wars: Episode I - The Phantom Menace,1999,PG,133,AVS,18
0076759,Star Wars: Episode IV - A New Hope,1977,PG,121,AVYS,19
0121766,Star Wars: Episode III - Revenge of the Sith,2005,PG-13,140,AVYS,44
0121765,Star Wars: Episode II - Attack of the Clones,2002,PG,143,AVS,81
0086190,Star Wars: Episode VI - Return of the Jedi,1983,PG,134,AVYS,83
0080684,Star Wars: Episode V - The Empire Strikes Back,1980,PG,124,AVYS,94
0796366,Star Trek,2009,PG-13,127,AVS,122


In [31]:
# Or, we could find out how many Movies that contained the letter "V" in genres:
# Presumably, this means "Adventure" (while A is for "Action", S is for "Sci Fy")

%%sql
SELECT COUNT(*) as "Number of Adventure Films" FROM Movie
  WHERE genre LIKE '%V%';

 * sqlite:////content/movie.sqlite
Done.


Number of Adventure Films
169


#8. Compound Conditions With OR, NOT, and AND

We can also use logical operators like OR, AND, and NOT to put together different conditions in the WHERE clause.

In [32]:
# Let's find out how many movies have "War" or "Star" somewhere in the title
%%sql

SELECT COUNT(*) AS "Either Star or War" FROM Movie 
  WHERE name LIke '%Star%'
  OR name LIKE '%War%';


 * sqlite:////content/movie.sqlite
Done.


Either Star or War
22


In [33]:
# Let's find out how many movies have "War" AND "Star" somewhere in the title
%%sql

SELECT COUNT(*) as "Both Star and War" FROM Movie 
  WHERE name LIke '%Star%'
  AND name LIKE '%War%';

 * sqlite:////content/movie.sqlite
Done.


Both Star and War
11


In [34]:
# Let's find out how many movies have NEITHER "War" NOR "Star" anywhere in the title
%%sql

SELECT COUNT(*) AS "Neither Star nor War" FROM Movie 
  WHERE name NOT LIke '%Star%'
  AND name NOT LIKE '%War%';

 * sqlite:////content/movie.sqlite
Done.


Neither Star nor War
705


#9. Tables JOINs and table aliases
Finally we can JOIN our tables together, using shared attributes. The stucture is as follows:

```
SELECT a,b,c FROM table1 JOIN table2
  ON table1.attribute_name = table2.attribute_name
```



In [35]:
# Let's start with a simple join of movies and Oscars
%%sql
SELECT * FROM Movie JOIN Oscar
  ON Movie.id = Oscar.movie_id
  LIMIT 10;


 * sqlite:////content/movie.sqlite
Done.


id,name,year,rating,runtime,genre,earnings_rank,movie_id,person_id,type,year_1
6751668,Parasite,2019,R,132,CDT,None,6751668,None,BEST-PICTURE,2020
7286456,Joker,2019,R,122,DT,62,7286456,0001618,BEST-ACTOR,2020
7549996,Judy,2019,PG-13,118,DR,None,7549996,0000250,BEST-ACTRESS,2020
7131622,Once Upon a Time... In Hollywood,2019,R,161,CD,None,7131622,0000093,BEST-SUPPORTING-ACTOR,2020
7653254,Marriage Story,2019,R,137,CDR,None,7653254,0000368,BEST-SUPPORTING-ACTRESS,2020
6751668,Parasite,2019,R,132,CDT,None,6751668,0094435,BEST-DIRECTOR,2020
6966692,Green Book,2018,PG-13,130,CD,None,6966692,None,BEST-PICTURE,2019
1727824,Bohemian Rhapsody,2018,PG-13,134,D,181,1727824,1785339,BEST-ACTOR,2019
5083738,The Favourite,2018,R,119,D,None,5083738,1469236,BEST-ACTRESS,2019
6966692,Green Book,2018,PG-13,130,CD,None,6966692,0991810,BEST-SUPPORTING-ACTOR,2019


In [36]:
# We can use this to ask question, such as "Which actors named Mark have appeared in Stars Wars movies?"
# To make our lives easier, we can use table aliases to avoid retyping the table name again and again

%%sql
SELECT DISTINCT P.name FROM Person P 
  JOIN Actor A on A.actor_id = P.id
  JOIN Movie M on A.movie_id = M.id
  WHERE M.name LIKE "Star Wars%"
  AND P.name LIKE 'Mark%';


 * sqlite:////content/movie.sqlite
Done.


name
Mark Hamill


In [37]:
# Or, how about, "Can you tell me who won best director in 2010?"

%%sql
SELECT P.name FROM Person P
  JOIN Oscar O on O.person_id = P.id
  WHERE O.year = 2010
  AND O.type = 'BEST-DIRECTOR';

 * sqlite:////content/movie.sqlite
Done.


name
Kathryn Bigelow


##10. Simple Aggregate Functions
Along with COUNT, we can also use functions like:
1. Use MAX/MIN to find the maximum or minimum values in a column.
2. Use AVG to find the average value in  a column.
3. Use SUM to sum up the results in a column.

In [38]:
# Let's find the longest runtime in the database
%%sql
SELECT MAX(runtime) FROM Movie

 * sqlite:////content/movie.sqlite
Done.


MAX(runtime)
238


In [39]:
# Let's find the average runtime of a movies featuring
# Harrison Ford
%%sql
SELECT AVG(runtime) FROM Movie M
  JOIN Actor A on A.movie_id = M.id
  JOIN Person P on P.id = A.actor_id
  WHERE P.name = 'Harrison Ford'

 * sqlite:////content/movie.sqlite
Done.


AVG(runtime)
125.72727272727273


In [40]:
# Let's find the total length of all 
# Spider-Man movies in the database

%%sql
SELECT SUM(runtime) FROM Movie 
  WHERE name LIKE "Spider-Man%"

 * sqlite:////content/movie.sqlite
Done.


SUM(runtime)
650
